In [18]:
n = 1
L = 8*n + 4
t = 1
Ne = Int(L / 2)

6

In [19]:
#Hamiltonian
function Hamiltonian(L,t)
    H = zeros(2*L, 2*L)
    H[1,L] = -t
    H[L,1] = -t
    H[2L,L+1] = -t
    H[L+1,2L] = -t
    for i=1:2*L
      for j=1:2*L
        if i == j + 1 || i == j - 1
          H[i,j] = -t
        end 
      end
    end
    H[L,L+1]=0
    H[L+1,L]=0
    return H
end

H = Hamiltonian(L,t);

In [20]:
#generate, convert state
using StatsBase

function Lstate(L)                                 #generate a state of length L
    Lstated, Lstateu, Lstate = zeros(L), zeros(L), zeros(L)
    randomposu = sample(collect(1:L), Int(L/4), replace=false)
    randomposd = sample(collect(1:L), Int(L/4), replace=false)
    for j in randomposu
        Lstateu[j] = 1
    end
    for j in randomposd
        Lstated[j] = -1
    end
    for i=1:L 
        if Lstated[i] == -1 && Lstateu[i] == 1
            Lstate[i] = 2
        else
            Lstate[i] = Lstated[i] + Lstateu[i]
        end
    end
    return Lstate
end

function extendedstate(Lstatevector)               #take a state of length L and make it 2*L
    L = length(Lstatevector)
    State2L = zeros(2*L)
    for j=1:L
        if Lstatevector[j] == -1
            State2L[j + L] = 1
        elseif Lstatevector[j] == 1
            State2L[j] = 1
        elseif Lstatevector[j] == 2
            State2L[j], State2L[j + L] = 1,1
        else
            State2L[j] = 0
        end
    end
    return State2L
end

function shortstate(ext_state)                    #pass trom 2*L state to L state
    L = Int(length(ext_state)/2)
    sh_state = zeros(L)
    for j=1:L
        if ext_state[j]==1
            sh_state[j]=1
        end
    end
    for j=1:L
        if ext_state[L+j]==1
            if sh_state[j]==1
                sh_state[j]=2
            else
                sh_state[j]=-1
            end
        end
    end
    return sh_state
end


shortstate (generic function with 1 method)

In [288]:
#generate a move
function Pauli(x,i,j)                 #check if state i and j are both occupied
    if x[i] == 1 && x[j] == 1
        return true
    end
    return false
end 

function occupied_states(v)         #generate a vector with positions of occupied states (from 2*L states)
    occ_states = []
    for i = 1:2*L
        if v[i] == 1
            append!(occ_states, i)
        end
    end
    return occ_states
end


function occupied_states_L(v)
    occ_states = []
    L =length(v)
    for i = 1 : L
        if abs(v[i]) == 1
            append!(occ_states, i )
            
        elseif v[i] == 2
            append!(occ_states, i )
        
        end
    end
    return occ_states
end


function move_L(w)
    
    v = copy(w)
    L = length(v)
    i = rand(occupied_states_L(v))
    j = rand(-1:2:1)

    j_pbc = mod(L+i+j,L)
    j_pbc == 0 ? j_pbc = L : j_pbc = j_pbc

        if v[i] != v[j_pbc] && v[j_pbc]!=2

            if abs(v[i])==1
            v[j_pbc] == 0 ? v[j_pbc] = v[i] : v[j_pbc] = 2
            v[i] = 0

            elseif v[i]==2    
            spin = rand(-1:2:1)

                if v[j_pbc] != spin
                v[i] = -spin

                    if abs(v[j_pbc]) == 1 
                        v[j_pbc] = 2 
                    elseif v[j_pbc] == 0 
                        v[j_pbc] = spin 
                    end

                end    
            end
            
        end

    return v, i, j_pbc
    
end


move_L (generic function with 1 method)

In [22]:
#generate unitary matrices and the ratio of determinants
using LinearAlgebra
U = eigvecs(H)  #unitary matrix
Udag = inv(U)   #U^-1
E = eigvals(H)  #eigenvalues of H

function U1(A, Ne) #take the first Ne columns
    U_1 = A[:,1:Ne]
    return U_1
end

U_1 = U1(U,Ne)

function U_tilde(A,v)                  #takes just the rows related to occupied states, Ne*Ne matrix
    U_tild = []
    U_tild = A[occupied_states(v),:]
    return U_tild
end

function ratio_det(v, z)
    rat= det(U_tilde(U_1,z))/det(U_tilde(U_1,v))
    return rat
end  


ratio_det (generic function with 1 method)

In [102]:
#Jastrow factor
function n_i(vi)           #counts the number of els in positions vi, returning (# up , # down)
    
    n_up = 0
    n_down = 0
    
        if vi == 1
            n_up += 1
            
        elseif vi == -1
            n_down += 1
                
        elseif vi == 2
            n_down += 1
            n_up += 1
                
        end

return n_up, n_down

end

function Jastrow(state,g,v)
            
    D=length(state)       
    
    Spins=0
    Occupancies=0        
    n1 = n_i(state[D])[1] + n_i(state[D])[2]
    n2 = 0
    
    for i = 1 : D
                
        n_up= n_i(state[i])[1]    
        n_down = n_i(state[i])[2]
                
    Spins = Spins + n_up*n_down 
                
    n2= n_up + n_down
    n1n2 = n1*n2   
    n1 = n2
                
    Occupancies = Occupancies + n1n2
            
        end
    
    arg=  -g*Spins -v*Occupancies
            
return exp(arg)

end

Jastrow (generic function with 1 method)

In [289]:
function T_Jastrow(x,j,v,g)
            
    L=length(x)   
    
    Sum=0
    
    ni = 0
    vij = 0
    
    for i in 1:L
        
    i_pbc = mod(L+i,L)
    i_pbc == 0 ? i_pbc = L : i_pbc = i_pbc
        
        ni = n_i(x[i_pbc])
        
        if i_pbc == j
             
              vji = 2*v
        elseif i_pbc == j + 1
            
              vji = 2*g      
        else 
               vji = 0  
        end
        
      Sum += vji * sum(ni)
        
    end
    
    return Sum
    
end

T_Jastrow (generic function with 1 method)

In [290]:
function ratio_Jastrow(x, xprime, i, j, g, v)    
    
    Tx= T_Jastrow(x,i,v,g)
    Txprime = T_Jastrow(xprime,j,v,g)
    
    ratio = (exp(-Txprime)/exp(-Tx))*(exp(2v-2g))
    
return ratio

end

ratio_Jastrow (generic function with 1 method)

In [296]:
v= Lstate(L)
w,i,j = hopping(v)

ratio_Jastrow(v, w, i, j, 10, 8)



0.01831563888873418

In [323]:
v= Lstate(L)
w,i,j = hopping(v)
T_Jastrow(v,1,10,8)

0

In [322]:
w,i,j = hopping(v)
ratio_Jastrow(v, w, i, j, 10, 8)

2.061153622438558e-9

In [324]:
    i_pbc = mod(j+L,L)
    i_pbc == 0 ? i_pbc = L : i_pbc = i_pbc
        

6

In [107]:
#This function calculates the density observable in the odd sites A
function nA(v)
    nA = 0
    for j=1:length(v)
        if j%2 != 0
            nA += sum(n_i(v[j])) 
        end
    end
    return nA*2/length(v)
end

#This function calculates the density observable in the even sites B
function nB(v)
    nB = 0
    for j=1:length(v)
        if j%2 == 0
            nB += sum(n_i(v[j])) 
        end
    end
    return nB*2/length(v)
end

nB (generic function with 1 method)

In [108]:
function hamiltonianelements(state1,state2,U,V)
    
    if state1 == state2
        D = length(state1)
    
        Spins = 0
        Occupancies = 0
        n1 = n_i(state1[D])[1] + n_i(state1[D])[2]
        n2 = 0
    
        for i = 1:D                
            n_up= n_i(state1[i])[1]
            n_down = n_i(state1[i])[2]
                
            Spins = Spins + n_up*n_down

            n2= n_up + n_down
            n1n2 = n1*n2
            n1 = n2

            Occupancies = Occupancies + n1n2    
        end
    
        arg = U*Spins + V*Occupancies
            
    return arg
        
    else
    
    w = abs.(abs.(state1) - abs.(state2))
    wr = occupied_states_L(w)
        
        if length(wr) == 2 && wr[2] - wr[1] == 1
            return -t
        else
            return 0
        end
    end
end

hamiltonianelements (generic function with 1 method)

In [327]:
function orthogonal(w, i)
    v = copy(w)
    L = length(v)
    states = []
    J = [-1,1]
    
    for j in J
        
        v = copy(w)
    
        j_pbc = mod(i+j,L)
        j_pbc == 0 ? j_pbc = L : j_pbc = j_pbc

            if v[i] != v[j_pbc] && v[j_pbc]!=2

                if abs(v[i])==1
                v[j_pbc] == 0 ? v[j_pbc] = v[i] : v[j_pbc] = 2 
                v[i] = 0    
                push!(states,v)

                elseif v[i]==2    
                Spin = [-1,1]

                    for spin in Spin
                    v = copy(w)

                            if spin != v[j_pbc] && v[j_pbc]!=2
                                v[j_pbc] == 0 ? v[j_pbc] = spin : v[j_pbc] = 2 
                                v[i] = -spin
                                push!(states,v)
                            end  
                      end
                 end   
          end
        
    end 
    
    return states
    
end

orthogonal (generic function with 1 method)

In [336]:
v = Lstate(12)

12-element Array{Float64,1}:
 -1.0
  0.0
  2.0
  0.0
  1.0
 -1.0
  0.0
  1.0
  0.0
  0.0
  0.0
  0.0

In [337]:
orthogonal(v, 3)

4-element Array{Any,1}:
 [-1.0, -1.0, 1.0, 0.0, 1.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
 [-1.0, 1.0, -1.0, 0.0, 1.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
 [-1.0, 0.0, 1.0, -1.0, 1.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
 [-1.0, 0.0, -1.0, 1.0, 1.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

In [110]:
import Random

function MC_en(state0, g, v, U,V, nsteps = 10000)
    Random.seed!(1234)
    state0_2L = extendedstate(state0)
    accepted_moves = 0
    local_est = 0
    state_L = copy(state0)
    local_est_array = []
    locals = []
    x_prime_L = []
    x_prime_2L_ = []
    for istep = 1:nsteps
        e_n = 0
        
        for j=1:Ne
        state_2L = extendedstate(state_L)
            state_new_L = move_L(state_L)
            state_new_2L = extendedstate(state_new_L)

            acc =  (ratio_det(state_2L,state_new_2L) * (Jastrow(state_new_L,g,v)/Jastrow(state_L,g,v)))^2       #acceptance 
            #display(acc)
            acc = min(acc,1)
                
            if acc>rand()        #metropolis rule
                x_L = state_new_L
                x_2L = state_new_2L
                accepted_moves += 1
                pot = log(Jastrow(x_L,-U,-V))
                for i in occupied_states_L(state_new_L)
                    o_states = orthogonal(state_new_L, i)

                    for j in 1:length(o_states)
                        x_prime_L = o_states[j]
                        x_prime_2L = extendedstate(x_prime_L)        

                        local_est = -t*ratio_det(x_2L,x_prime_2L)*(Jastrow(x_prime_L,g,v)/Jastrow(x_L,g,v))
                        e_n += local_est
                     end
                    
                end
                e_n += pot
                push!(locals,e_n)
                state_L = x_L
            end         
        end
    end
    return mean(locals)
end 



MC_en (generic function with 2 methods)

In [111]:
a = Lstate(L)

MC_prova= MC_en(a,10,8,30,5,10000)

LoadError: [91mBoundsError: attempt to access 6-element Array{Float64,1} at index [7][39m

In [112]:
using Plots

yA = zeros(0)
yB = zeros(0)
v = collect(1:0.5:10)
for x in v
    MC_prova= MC_en(a,10,x,30,5,10000)
    append!(yA, MC_prova)
end
plot(v,yA)

LoadError: InitError: [91mInterruptException:[39m
during initialization of module Ogg_jll

In [113]:
import Random

function MC_en(state0, g, v, U, V, nsteps = 1000)
    Random.seed!(1234)
    
    accepted_moves = 0
    local_est = 0
    state_L = copy(state0)
    local_est_array = []
    locals = []
    x_prime_L = []
    x_prime_2L = []
    
    for istep = 1:nsteps  
        
        e_n = 0
        state_2L = extendedstate(state_L)
        state_new_L = move_last_2L(state_2L)
        state_new_2L = extendedstate(state_new_L)

        acc =  (ratio_det(state_2L,state_new_2L) * (Jastrow(state_new_L,g,v)/Jastrow(state_L,g,v)))^2       #acceptance 
        #display(acc)
        acc = min(acc,1)
                
        if acc>rand()        #metropolis rule
             x_L = copy(state_new_L)
             #display(x_L)
             x_2L = copy(state_new_2L)
             accepted_moves += 1
             pot = log(Jastrow(x_L,-U,-V))
             #display(pot)
            
             for i in occupied_states_L(x_L)
                 o_states = orthogonal(x_L, i)

                 for j in 1:length(o_states)
                        x_prime_L = o_states[j]
                        x_prime_2L = extendedstate(x_prime_L)        

                        local_est = -t*ratio_det(x_2L,x_prime_2L)*(Jastrow(x_prime_L,g,v)/Jastrow(x_L,g,v))
                        display(local_est)
                        e_n += local_est
                 end
                    
            end
            
            #display(e_n)
            e_n += pot
            
            push!(locals,e_n)
            state_L = copy(x_L)
        end         
        
    end
    return mean(locals)
end

MC_en (generic function with 2 methods)

In [137]:
function hopping(v)
    w = copy(v)
    occupied_positions = occupied_states_L(v)
    i = rand(occupied_positions)
    nn = [v[i + 1],v[i - 1]]
    nnpos = [i + 1, i - 1]
    rndnnpos = rand([i + 1,i - 1])
    rndnn = rand(nn)
    
    if nn[1] == 2 && nn[2] == 2 || v[i] == 1 && nn[1] == 1 && nn[2] == 2 || v[i] == 1 && nn[1] == 2 && nn[2] == 1 || v[i] == 1 && nn[1] == 1 && nn[2] == 1 || v[i] == -1 && nn[1] == -1 && nn[2] == 2 || v[i] == -1 && nn[1] == 2 && nn[2] == -1 || v[i] == -1 && nn[1] == -1 && nn[2] == -1
        i = rand(deleteat!(occupied_positions, findall(x->x==i,occupied_positions)))
    end
    
    if v[i] == 1 && v[rndnnpos] == 1 || v[i] == 1 && v[rndnnpos] == 2
        rndnnpos = rand(deleteat!(nnpos, findall(x->x==rndnnpos, nnpos)))
    end
    
    if v[i] == -1 && v[rndnnpos] == -1 || v[i] == -1 && v[rndnnpos] == 2
        rndnnpos = rand(deleteat!(nnpos, findall(x->x==rndnnpos, nnpos)))
    end
    
    if v[i] == 2 && v[rndnnpos] == 2
        rndnnpos = rand(deleteat!(nnpos, findall(x->x==rndnnpos, nnpos)))
    end
     
    if v[i] == 1
        if v[rndnnpos] == -1
            w[i], w[rndnnpos] = 0, 2
        elseif v[rndnnpos] == 0
            w[i], w[rndnnpos] = 0, 1
        end
    
    elseif v[i] == -1 
        if v[rndnnpos] == 1
            w[i], w[rndnnpos] = 0, 2
        elseif v[rndnnpos] == 0
            w[i], w[rndnnpos] = 0, -1
        end
    
    elseif v[i] == 2
        if v[rndnnpos] == 1 
            w[i], w[rndnnpos] = 1, 2
        elseif v[rndnnpos] == -1
            w[i], w[rndnnpos] = -1,2
        elseif v[rndnnpos] == 0
            choosenspin = rand([1,-1])
            w[i], w[rndnnpos] = -1*choosenspin, choosenspin
        end
    end
    
    return w,  i , rndnnpos
end

hopping (generic function with 1 method)

In [363]:
#Monte Carlo simulation that calculates nA along the simulation
import Random

function MC_pietro(q0, g, v, U,V, nsteps = 1000)
    #Random.seed!(1234)
    local_est = 0
    state_L = copy(q0)
    local_est_array = []
    locals = []
    
    locals2 = []
    
    x_prime_L = []
    x_prime_2L = []
    sumA=0
    sumB=0
    accepted_moves = 0
    state_new_L = zeros()
    x_L = zeros()
    x_2L = zeros()
    
    Suma_A = []
    Suma_B = []
    
    k=0
    l=0
    
    
    for istep = 1:nsteps
        e_n = 0

        for j = 1:Ne
            state_2L = extendedstate(state_L)
            state_new_L,k,l = hopping(state_L)
            state_new_2L = extendedstate(state_new_L)
            acc =  (ratio_det(state_2L,state_new_2L) * (ratio_Jastrow(state_L, state_new_L, k, l, g, v)))^2       #acceptance 

            acc = min(1,acc)

            if acc>rand()        #metropolis rule
                x_L = copy(state_new_L)
                #display(x_L)
                x_2L = copy(state_new_2L)
                accepted_moves += 1
            end
        end
       #= 
        n_newA= nA(state_new_L)
        sumA+=n_newA
        push!(Suma_A,n_newA)
                #push!(Suma_A,n_newA)

        n_newB= nB(state_new_L)
        sumB+=n_newB
        push!(Suma_B,n_newB)
                #push!(Suma_B,n_newB)
    
        =#
        
        pot = log(Jastrow(x_L,-U,-V))
        #display(pot)
  
        
                 for i in occupied_states_L(x_L)
        
                 o_states = orthogonal(x_L, i)
            
                     for j in 1:length(o_states)
                            x_prime_L = o_states[j]
                            x_prime_2L = extendedstate(x_prime_L)        

                            local_est = -t*ratio_det(x_2L,x_prime_2L)*(ratio_Jastrow(x_L, x_prime_L, k, l, g, v)) 

                          #  display(ratio_Jastrow(x_L, x_prime_L, k, l, g, v)) 

                           # local_est = -t*ratio_det(x_2L,x_prime_2L)
                            e_n += local_est
                     end      
                 end
   
        
           e_n += pot 
        
           push!(locals,e_n)
        
           state_L = copy(x_L)
          end
    return  mean(locals)
end 

MC_pietro (generic function with 2 methods)

In [373]:
a = Lstate(L)
MC_prova= MC_pietro(a,10,5,30,5,100)

In [300]:
using Plots
x = collect(2:1:8)
y = [MC_pietro(a,10,v,30,5,100) for v in x]

plot(x,y)

LoadError: [91mBoundsError: attempt to access 12-element Array{Float64,1} at index [0][39m